In [ ]:
# uncomment if running in vs code
# %cd ..
# %cd ..
# %cd ..

## Kernel: run_compound

Compare times of a priori and a posteriori approach

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import Notebooks.Scripts.post_process as post

In [3]:
class_list = pd.read_csv('fitted_stats/classified.csv', index_col=0).columns.to_list()
sub_count_list = pd.read_csv('fitted_stats/sub_county.csv', index_col=0).columns.to_list()

MDA + scatter times

In [4]:
times = pd.read_csv('Models/MDA_2d/times.csv', index_col = 0)
clock = len(times[times[times.columns[0]] != 0])
times = times.iloc[:clock]

In [ ]:
times.sum().sum()/60

In [6]:
tgp_folder = 'Notebooks/4.Active_learning/'

## TGP on single output

In [ ]:
diff_comp_single, samples_req_single = post.collect_tgp_times(tgp_folder)
samples_req_single

In [ ]:
diff_comp_single

In [ ]:
diff_comp_single.sum()

In [ ]:
single_comp = diff_comp_single[['A Posteriori (2 dims)']].copy(deep = True)
single_comp.rename(columns = {'A Posteriori (2 dims)': 'Active learning'}, inplace = True)
single_comp.loc[:, 'Equidistant sampling'] = times.sum()/60
col_ind = [1, 0]
post.plot_times(single_comp.iloc[:, col_ind])
# plt.savefig('Figures/PDF/f03.pdf', dpi=300, format='pdf', bbox_inches="tight")
# plt.savefig('Figures/PNG/f03.png', format='png', bbox_inches="tight")

## TGP on classified outputs

In [16]:
class_list = pd.read_csv('fitted_stats/classified.csv', index_col=0).columns.to_list()

In [ ]:
diff_comp_multi, samp_multi = post.collect_tgp_times(tgp_folder, classif = class_list)
samp_multi

In [ ]:
diff_comp_multi

In [ ]:
diff_comp_multi.sum()

## TGP on sub-county model

In [ ]:
sub_count_list = pd.read_csv('fitted_stats/sub_county.csv', index_col=0).columns.to_list()
diff_comp_sub, samp_sub = post.collect_tgp_times(tgp_folder, classif = sub_count_list, dimensions=[2])
samp_sub

In [ ]:
diff_comp_sub

In [ ]:
diff_comp_sub.sum()

In [ ]:
diff_comp_multi

In [56]:
def plot_times_all(df_prior, df_single, df_comp, df_sub):
    fig, ax = plt.subplots(1, 4, figsize = (8, 6), gridspec_kw={'width_ratios': [0.8, 5, 5, 0.8]}, sharey=True)
    list = [df_prior, df_single, df_comp, df_sub]
    names = ['Equidistant\nsampling', 'Active learning\ncomplete', 'Active learning\nclassified', 'Active learning\nsub-county']
    for i in range(4):
        df = list[i]
        print(df)
        num_bars = df.shape[1]
        x_values = np.arange(1, num_bars + 1, 1)
        print(num_bars)
        print(i)
        ax[i].bar(x_values, df.loc['FIAT'].values, label='Delft-FIAT')
        ax[i].bar(x_values, df.loc['SFINCS'].values, bottom=df.loc['FIAT'].values, label='SFINCS')
        ax[i].set_xticks(x_values, df.columns, fontsize = 12)
        ax[i].grid()
        ax[i].axhline(df_prior.sum().item(), c = 'r', lw = 2, label = 'Equidistant sampling\ncomputational time', zorder=0)

        if i > 0:
            for j in range(num_bars):
                if j == 0:
                    label = 'TGP-LLM'
                else:
                    label = ''
                ax[i].bar(x_values[j], df.loc['TGP'].iloc[j], bottom=[df.loc['FIAT'].iloc[j]+df.loc['SFINCS'].iloc[j]], label=label, color= 'k')



        ax[i].set_xlabel(names[i], fontsize = 12, fontweight = 'bold')
    ax[0].set_ylabel('Time [min]', fontsize = 12)
    # plt.title('Comparing Computational Cost of Sampling Algroithms for Complete', fontsize = 12)

    ax[1].legend(fontsize = 12)
    fig.text(0.55, -0.04, 'Number of dimensions for different approaches and number of geographic locations [-]   ', ha='center', fontsize=14)
    return None

In [57]:
Prior = pd.DataFrame(times.sum()/60, columns = ['2'])

In [ ]:
new = diff_comp_multi.copy(deep = True)
diff_comp_single = diff_comp_single.rename(columns={'A Posteriori (2 dims)': '2',
                                                    'A Posteriori (3 dims)': '3',
                                                    'A Posteriori (4 dims)': '4',
                                                    'A Posteriori (5 dims)': '5',
                                                    'A Posteriori (6 dims)': '6'})
new = new.rename(columns={'A Posteriori (2 dims)': '2',
                          'A Posteriori (3 dims)': '3',
                          'A Posteriori (4 dims)': '4',
                          'A Posteriori (5 dims)': '5',
                          'A Posteriori (6 dims)': '6'})
diff_comp_sub = diff_comp_sub.rename(columns={'A Posteriori (2 dims)': '2'})
plot_times_all(Prior, diff_comp_single, new, diff_comp_sub)
plt.tight_layout(pad = 0.0000000001, rect = (0, 0, 1, 1))
plt.savefig('Figures/PDF/f04.pdf', dpi=300, format='pdf', bbox_inches="tight")
plt.savefig('Figures/PNG/f04.png', format='png', bbox_inches="tight")